### 

# Pre-Proceso: preparación Transaccional + EDA

## Importar paquetes

In [1]:
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
import math
import seaborn as sns
%matplotlib inline

letter = string.ascii_uppercase

## Función: Importar Excel

In [1]:
# Borrar filas y columnas vacías ENTREMEDIO de la tabla importada (agregar dropna después de importar el excel)

def importar_excel(nombre_excel, xlsx=True):
    
    x = pd.ExcelFile(nombre_excel)
    y = x.sheet_names
    dict = {}
    for i, j in zip(range(0,len(y)),y):
        dict.update({i : j})
    
    hoja = int(input(f'¿Cuál de las siguientes hojas quieres importar? (Selecciona el número de la hoja): \n {dict} \n '))
    
    
    if xlsx is True:   
        nombre_csv = nombre_excel[0:-5]
    else: 
        nombre_csv = nombre_excel[0:-4]
        
    df1 = pd.read_excel(nombre_excel, sheet_name=hoja)
    
    def get_first_row(df):
        for index, row in df.iterrows():
            if not row.isnull().values.all():
                if index != 0:
                    return index + 1
                else:
                    return index
            
    def get_start_column(df):
        for i, column in enumerate(df.columns):
            if df[column].first_valid_index():
                return letter[i]
            elif df.iloc[0].first_valid_index():
                return 'A'
    
    def get_last_column(df):
        for i, column in enumerate(df.columns):
            if get_start_column(df):
                if len(df.columns)-i <=25:
                    return letter[len(df.columns) - i]
                else:
                    return f"A{letter[(len(df.columns) - i)-25]}"

    def usecols(df):
        start = get_start_column(df)
        end = get_last_column(df)
        return f"{start}:{end}"

    df = pd.read_excel(nombre_excel, sheet_name=hoja, header=get_first_row(df1), usecols=usecols(df1))
    archivo_csv = df.to_csv(f"{nombre_csv}.csv",
                            index = None,
                            header = True)
    archivo_csv = pd.DataFrame(pd.read_csv(f"{nombre_csv}.csv"))
    
    return archivo_csv

### Usar función para importar archivo excel y asignar a dataframe "orig"

In [ ]:
orig = importar_excel('ruta de acceso archivo excel', xlsx=False);orig.head()

### Importar archivo .csv y asignar a dataframe "orig"

In [ ]:
orig = pd.read_csv('ruta de acceso archivo .csv',
                   delimiter=',');orig.head()

## Función: Overview

In [2]:
# Permite ver valores únicos por campo, nulos y no-nulos, formato, máximo y mínimo.
# Además un resumen estadístico con los rangos y distribución de la data

def overview(dataframe):
    
    data_resumen = {'Valores Unicos':dataframe.nunique(),'No-Nulos': dataframe.notnull().sum(), 
                    'Nulos': dataframe.isnull().sum(), 'Formato': dataframe.dtypes, 'Min': dataframe.min(),
                    'Max': dataframe.max()}
    resumen = pd.DataFrame(data=data_resumen)
    print(dataframe.describe())
    
    return resumen

### Usar función para un resumen del dataframe

In [ ]:
overview(orig)

In [ ]:
orig.info()

### Para unir verticalmente dos dataframes con los mismos campos

In [ ]:
# Para unir verticalmente dos dataframes
trs = pd.concat([df1, df2], axis=0)

### Para filtrar un dataframe usando otro con las características, y crear nuevo dataframe con filtro aplicado

In [ ]:
# Para unir dos dataframes con columna en comun
common = trs.merge(df_filtro,on=['campo_en_comun'])
df_filtrado = trs[(df.material.isin(common.campo_en_comun))]

### Para filtrar un dataframe y solo ver un SKU en específico

In [ ]:
trs[trs['sku'] == 10001266]

## Función: Esquema Transaccional

In [3]:
# Se define una lista con los índices del dataframe junto con los nombres de los campos
# para armar dataframe nuevo (transaccional)

def esquema_transaccional(df,index_campos,_1='fecha', _2='id_cliente', _3='nro_transaccion', 
                          _4='sku',_5='cantidad', _6='unidadmedida', _7=None, _8=None):

    campos = df.columns.values[index_campos].tolist()
    columnas = [_1,_2,_3,_4,_5,_6,_7,_8]
    transaccional = []

    for (campo,columna) in zip(campos,columnas):
        df0 = df.rename(columns=dict(zip(campos, columnas)))
        transaccional.append(columna)
        
    df1 = df0[transaccional]
    
    return df1

### Usar función para armar dataframe transaccional "trs"

In [ ]:
trs = esquema_transaccional(orig,[0,1,2,3,4,5]);trs

## Función: Nulos

In [4]:
def nulos(df):

    is_NaN = df.isnull()
    row_has_NaN = is_NaN.any(axis=1)
    rows_with_NaN = df[row_has_NaN]
    n_nulos = len(rows_with_NaN)
    
    if n_nulos > 0:
        
        print(f'¡Se han encontrado {n_nulos} registros que contienen valores nulos!\n')
        
        dict = {'Analizar': 1, 'Borrar': 0}
        answer = int(input(f'¿Qué quieres hacer con estos registros? (Escribe el número de la respuesta): \n {dict} \n '))
        
        if answer == 0:
            df.dropna(inplace=True)
            return df
        
        else:
            print(rows_with_NaN.nunique())
            return rows_with_NaN
        
    else:
        print(f'¡No se encontraron registros nulos!')

### Usar función para revisar registros con celdas nulas

In [ ]:
nulos(trs)

## Función: Duplicados

In [2]:
def duplicados(df):
    
    duplicateDFRow = df[df.duplicated()]
    n_duplicados = len(duplicateDFRow)
    
    if n_duplicados > 0:

        print(f'¡Se han encontrado {n_duplicados} registros duplicados!')
        dict = {'Analizar': 1, 'Borrar': 0}
        answer = int(input(f'¿Qué quieres hacer con estos registros? (Escribe el número de la respuesta): \n {dict} \n '))
        
        if answer == 0:
            df1 = df
            df1.drop_duplicates(keep='first',inplace=True)
            return df1
        
        else:
            print(duplicateDFRow.nunique())
            return duplicateDFRow
        
    else:
        print(f'¡No se encontraron registros duplicados!')

### Usar función para revisar y/o eliminar registros duplicados

In [ ]:
duplicados(trs)

## Función: Columna Fecha

In [ ]:
def col_fecha(df,columna_fecha, texto=False):
    
    from datetime import datetime
    
    # Opción para casos donde la fecha esté separada en columnas de año y mes, para armar columna mensual
    if texto == True:
        month = []
        for i in range (0,len(df)):
            mes = df.iloc[i,1]
    
            if mes == 'ENERO':
                mes_i = '01'
            elif mes == 'FEBRERO':
                mes_i = '02'
            elif mes == 'MARZO':
                mes_i = '03'
            elif mes == 'ABRIL':
                mes_i = '04'
            elif mes == 'MAYO':
                mes_i = '05'
            elif mes == 'JUNIO':
                mes_i = '06'
            elif mes == 'JULIO':
                mes_i = '07'
            elif mes == 'AGOSTO':
                mes_i = '08'
            elif mes == 'SEPTIEMBRE':
                mes_i = '09'
            elif mes == 'OCTUBRE':
                mes_i = '10'
            elif mes == 'NOVIEMBRE':
                mes_i = '11'
            elif mes == 'DICIEMBRE':
                mes_i = '12'

            month.append(mes_i)
    
        df['mes'] = month
        df['año'] = df['año'].apply(str)
        
        # Se especifica si existe un campo con el día para usarlo en la construcción de la fecha, o se deja "01" por defecto
        if 'dia' in df.columns:
            df['fecha'] = df['año'] + '-' + df['mes'] + df['dia'].apply(str)
        else:
            df['fecha'] = df['año'] + '-' + df['mes'] + '-01'
    
    # Deja la nueva columna de fecha en el indice 0 (por motivos de orden de la serie de tiempo)
    cols = list(df)
    cols.insert(0, cols.pop(cols.index('fecha')))
    df = df.loc[:, cols]

    df[columna_fecha] = df[columna_fecha].str.slice(stop=10)

    date = []
    for i in range(0,len(df)):
        
        fecha_i = df.iloc[i,0]
        if fecha_i[2] == '-':
            
            if fecha_i[-3] == '-':
                fecha_i = pd.to_datetime(fecha_i, format = '%d-%m-%y')
            else:
                fecha_i = pd.to_datetime(fecha_i, format = '%d-%m-%Y')
           
        else:
            fecha_i = pd.to_datetime(fecha_i, format = '%Y-%m-%d')
     
        month_i = fecha_i.month
        year_i = fecha_i.year
        day_i = fecha_i.day
        fecha_i2 = f'{year_i}-'+f'{month_i}-'+f'{day_i}'
            
        date.append(fecha_i2)

    df[columna_fecha] = date
    df[columna_fecha] = pd.to_datetime(df[columna_fecha], format = '%Y-%m-%d')
    df.sort_values(by=[columna_fecha], inplace = True, ascending = False)
    
    return df

### Usar función para crear y/o formatear campo de fecha en transaccional

In [ ]:
trs = col_fecha(trs,'fecha');trs

## Función: Columna(s) Entero(s)

In [7]:
def enteros(df,_1='cantidad',_2=None):
    
    campos = [_1,_2]
    
    for x in campos:
        df[x].fillna(0, inplace = True)
        df[x] = df[x].astype(int)
    
    return df

### Usar función para formatear en enteros el/los campo(s) con valores a predecir

In [ ]:
trs = enteros(trs,'cantidad')

## Función: Runners del negocio

In [ ]:
def runners(df, semana=False): # semanal: 24. mensual: 12

    if semana == False:
        df1 = df.groupby(['sku',pd.Grouper(key='fecha', freq='MS')]).sum().reset_index()
        ventanas = 12
    else:    
        df1 = df.groupby(['sku',pd.Grouper(key='fecha', freq='W-MON')]).sum().reset_index()
        ventanas = 24
        
    df1.sort_values(by=['fecha'], ascending=False, inplace=True)
    df1['distancia'] = df1['fecha'].diff().dt.days.ne(0).cumsum()
    df1['peso'] = round((1/df1['distancia']),2)
    df1 = df1[df1['distancia'] <= ventanas]
    df1 = df1.groupby(['sku']).sum().reset_index()
    
    thresh = []
    for i in range(1, ventanas+1):
        x = 1/i
        thresh.append(x)
        
    x = round(sum(thresh),2)
    
    clasificacion = []
    
    for i in range(0,len(df1)):
        clas = df1.loc[df1.index[i], 'peso']

        if clas < x*0.25:
            clas_i = 'LR'
        elif (clas >= x*0.25 and clas < x*0.75):
            clas_i = 'MR'
        elif clas >= x*0.75:
            clas_i = 'HR'

        clasificacion.append(clas_i)

    df1['clasificacion'] = clasificacion
    print(df1['clasificacion'].value_counts())
    
    df1['participacion_Q'] = df1['cantidad'] * df1['peso']
    df1['participacion_$'] =(df1['ingreso_neto'] * df1['peso']).astype(int)
    
    df1 = df1[['sku','cantidad','ingreso_neto','clasificacion','participacion_Q','participacion_$']]
    
    return df1

### Usar función para detectar High, Medium y Low Runners del negocio, para últimos 12 meses o 24 semanas

runners(trs)

## Función: Outliers

In [ ]:
def outliers(df, factor = 2, muestra=10):

    df2 = df.groupby(['sku', pd.Grouper(key='fecha', freq='MS')])['cantidad'].sum().reset_index()
    df_principal = df2[['fecha', 'sku', 'cantidad']]
    df_principal = df2.rename(columns={'cantidad':'total_mes'})
    df_principal['mean'] = round(df_principal.groupby(['sku'])['total_mes'].transform('mean'),2)
    df_principal['std'] = round(df_principal.groupby(['sku'])['total_mes'].transform('std').fillna(0),2)
    
    valores = []
    df_grafico = pd.DataFrame(np.array([1, 2, 3, 4, 5]), columns=['factor'])
    
    for i in range(1,6):
        
        df_principal[f'threshold_{i}'] = (df_principal['mean'] + (i * df_principal['std'])).astype(int)
        df_principal[f'diff_{i}'] = (df_principal['total_mes'] - df_principal[f'threshold_{i}']).astype(int)
        n_outliers_i = len(df_principal[df_principal[f'diff_{i}'] > 0])
        valores.append(n_outliers_i)
        
    del df_principal['mean'] 
    del df_principal['std']
    
    df_grafico['n_outliers'] = valores
    print(df_grafico)
    
    plt.figure(figsize=(13,8))
    sns.set_theme(style="whitegrid")
    ax = sns.barplot(x=df_grafico['factor'], y=df_grafico['n_outliers'], palette="Blues_r", alpha=0.8)
    for index, row in df_grafico.iterrows():
        ax.text(row.name,row.n_outliers, int(row.n_outliers), color='black', ha="center")

    df_principal.sort_values(by=[f'diff_{factor}'], ascending=False, inplace=True)
    df_principal = df_principal[['fecha', 'total_mes',f'threshold_{factor}',f'diff_{factor}']]

    return df_principal.head(muestra)

### Usar función para ver nº de outliers según factor, dataframe con factor elegido y gráfico

In [ ]:
outliers(df, factor = 2, muestra = 10)

### Agrupar mensualmente las ventas

In [ ]:
trsmes = trs.groupby([pd.Grouper(key='fecha', freq='MS'),'id_cliente','canal','sku']).sum().reset_index()

## Función: Pareto del negocio (Ingresos)

In [1]:
def pareto_p(df):
    
    df1 = df.groupby('sku').sum().reset_index()
    del df1['precioVenta']
    total_ingresos = df1['ingreso_neto'].sum()
    df1['peso_$ (%)'] = round((df1['ingreso_neto'].div(total_ingresos,axis=0))*100,3)
    df1.sort_values(by=['peso_$ (%)'], ascending=False, inplace=True)
    df1['acumulado'] = df1['peso_$ (%)'].cumsum()
    df2 = df1[df1['acumulado'] <= 80.1]
    n_sku_pareto = df2['sku'].nunique()
    print(f'Número de SKUs que componen el 80% de los ingresos del negocio = {n_sku_pareto}')
    
    return df2

### Usar función para armar dataframe con el pareto del negocio, asignar a dataframe para ver resumen

In [ ]:
pareto_p = pareto_p(trs)

## Función: Pareto del negocio (Un. vendidas)

In [ ]:
def pareto_q(df):
    
    df1 = df.groupby('sku').sum().reset_index()
    del df1['precioVenta']
    total_cantidad = df1['cantidad'].sum()
    df1['peso_q (%)'] = round((df1['cantidad'].div(total_cantidad,axis=0))*100,3)
    df1.sort_values(by=['peso_q (%)'], ascending=False, inplace=True)
    df1['acumulado'] = df1['peso_q (%)'].cumsum()
    df2 = df1[df1['acumulado'] <= 80.1]
    n_sku_pareto = df2['sku'].nunique()
    print(f'Número de SKUs que componen el 80% de las unidades vendidas del negocio = {n_sku_pareto}')
    
    return df2

### Usar función para armar dataframe con el pareto del negocio, asignar a dataframe para ver resumen

In [ ]:
pareto_q = pareto_q(trs)

### Ordena de menor a mayor (True) y viceversa las unidades vendidas

In [ ]:
trs.sort_values(by=['cantidad'], ascending=True, inplace=False)

### Graficar comportamiento total de ventas

In [ ]:
# Comportamiento total de ventas
sns.set(rc={'figure.figsize':(15,10)})
sns.lineplot(data=trsmes.groupby(['fecha'])['cantidad'].sum())

## Función: Top 10 por agrupación - Cantidad

In [ ]:
# Top 10 por categoria
def ventas_top10(df,jerarquia,top=10):
    
    jer = df.groupby([jerarquia]).sum()
    jer = jer.sort_values(by='cantidad',ascending=False)
    jer1 = jer.nlargest(top, 'cantidad').reset_index()

    # plot
    plt.figure(figsize=(17,10))
    sns.set_theme(style="whitegrid")
    ax = sns.barplot(x=jer1[jerarquia], y=jer1['cantidad'], order=jer1[jerarquia],palette="Blues_r",alpha=0.8)
    for index, row in jer1.iterrows():
        ax.text(row.name,row.cantidad, int(row.cantidad), color='black', ha="center")
    
    jerarquia = jerarquia.capitalize()
    plt.title(f"Top {top} {jerarquia} - Volumen", fontsize=14)
    plt.ylabel("Unidades Vendidas", fontsize=14)
    plt.xlabel(f"{jerarquia}", fontsize=14)
    
    return plt.show()

### Usar función para graficar top 10 - Cantidad

In [ ]:
ventas_top10(trsmes,'jerarquia')

## Función: Top 10 por agrupación - Ingresos

In [ ]:
# Top 10 en ventas (si hay monto facturado)
def ingresos(df,jerarquia, top=10):
    
    jer = df.groupby([jerarquia]).sum()
    jer = jer.sort_values(by='ingreso_neto',ascending=False)
    jer1 = jer.nlargest(top, 'ingreso_neto').reset_index()

    # plot
    plt.figure(figsize=(17,10))
    sns.set_theme(style="whitegrid")
    ax = sns.barplot(x=jer1[jerarquia], y=jer1['ingreso_neto'], order=jer1[jerarquia],palette="Blues_r",alpha=0.8)
    for index, row in jer1.iterrows():
        ax.text(row.name,row.ingreso_neto, int(row.ingreso_neto), color='black', ha="center")
    
    jerarquia = jerarquia.capitalize()
    plt.title(f"Top {top} {jerarquia} - Ingresos Netos", fontsize=14)
    plt.ylabel("Monto en $", fontsize=14)
    plt.xlabel(f"{jerarquia}", fontsize=14)
    
    return plt.show()

### Usar función para graficar top 10 - Ingresos

In [ ]:
ingresos(trsmes,'canal')

In [ ]:
# Función que corre pero toma mucho tiempo, versión de prueba

def outliers1(sku='sku', fecha='fecha', cantidad='real', factores=[1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]):
    
    # Creación dataframe con totales por sku por fecha (al final se agregan thresholds a cada uno)
    
    data_df_principal = {'sku': df1[sku],'fecha_mes':df1[fecha], 'real': df1[cantidad]}
    dfp = pd.DataFrame(data=data_df_principal)
    df_principal = dfp.groupby(['sku', pd.Grouper(key='fecha_mes', freq='MS')]).sum()
    df_principal.reset_index(drop=False, inplace=True)
    
    # Construcción thresholds

    factores_prueba = factores
    n_outliers = []
    for factor in factores_prueba:
        
        df_thresholds = df_principal.groupby(['sku']).mean()
        df_thresholds['std'] = df_principal.groupby(['sku']).std()
        df_thresholds = df_thresholds.rename(columns={'real': 'mean'})
        df_thresholds['std'] = df_thresholds['std'].fillna(0)
        df_thresholds['threshold'] = df_thresholds['mean'] + (factor*df_thresholds['std'])
        df_thresholds.reset_index(drop=False, inplace=True)

        # Tomar thresholds y armar la columna para df_principal, para comparar

        valores = []

        for k in range(0, len(df_thresholds)):
            threshold_valor = df_thresholds.iloc[k,3]

            for i in df_principal['sku']:
                o = df_thresholds.iloc[k,0]            

                if i == o:
                    valores.append(threshold_valor)

        df_principal['threshold'] = valores

        outliers = []
        diff = []
        for j in range(0, len(df_principal)):
            real = df_principal.iloc[j,2]
            thresh = df_principal.iloc[j,3]
            dif = abs(real - thresh)

            diff.append(dif)

            if real > thresh:
                outliers.append(1)
            else:
                outliers.append(0)


        df_principal['outliers'] = outliers
        df_principal['outliers'] = df_principal['outliers'].astype(int)
        df_principal['diff'] = diff
        df_principal['diff'] = df_principal['diff'].astype(int)
        df_output = df_principal[df_principal['outliers'] == 1]
        del df_output['outliers']
        df_output.sort_values(by=['diff'], ascending=False)
        n_outliers.append(len(df_output))
        
        df_principal = dfp.groupby(['sku', pd.Grouper(key='fecha_mes', freq='MS')]).sum()
        df_principal.reset_index(drop=False, inplace=True)
        
    data_df_outliers = {'factor': factores_prueba,'n_outliers': n_outliers}
    df_outliers = pd.DataFrame(data=data_df_outliers)
    print(df_outliers)
    plt.plot(df_outliers['factor'],df_outliers['n_outliers'], color = 'blue', marker = 'o')
    plt.title('Outliers según Factor')
    plt.xlabel('Factor')
    plt.ylabel('Nº Outliers')
    plt.show()
    
    
    return df_outliers

## Al terminar de procesar transaccional, exportar dataframe a archivo .csv (UTF-8)

In [ ]:
# Para exportar dataframe de transaccional a un csv
trs.to_csv(r'/ruta_de_archivo/titulo_archivo_transaccional.csv', index=False)

# Post-Proceso: Transaccional + Inferencias

## Función: Métricas Error

In [3]:
# Para calcular smape individual por ventana de tiempo + otros cálculos generales, uniendo archivo de inferencias con transaccional

def metricas_error(df,inferencia='inference.csv', semanal = False, agrupacion=None):

    # importar archivo inferencias
    inferencia = pd.read_csv(inferencia)
    inferencia.sort_values(by=['item','date'], ascending=True, inplace=True)
    inferencia['date'] = pd.to_datetime(inferencia['date'], format = '%Y-%m-%d')

    # adaptar transaccional

    # Caso Mensual
    if semanal == False:
        
        # Agrupacion solo por sku
        if agrupacion == None:
            
            transaccional = df.groupby([pd.Grouper(key='fecha', freq='MS'), 'sku']).sum()
            transaccional.reset_index(drop=False, inplace=True)
            transaccional.sort_values(by=['sku','fecha'], ascending=True, inplace=True)
        
        # Agrupacion además de sku
        else:
            transaccional = df.groupby([pd.Grouper(key='fecha', freq='MS'), 'sku']).sum()
            transaccional.reset_index(drop=False, inplace=True)
            transaccional.sort_values(by=[agrupacion,'sku','fecha'], ascending=True, inplace=True)
            transaccional["item"] = transaccional[agrupacion].astype(str) + '|' + transaccional["sku"].astype(str)
            del transaccional['sku']
            del transaccional[agrupacion]
        
    # Caso semanal
    else:
        
        # Agrupacion solo por sku
        if agrupacion == None:
            
            transaccional = df.groupby([pd.Grouper(key='fecha', freq='W-MON'), 'sku']).sum()
            transaccional.reset_index(drop=False, inplace=True)
            transaccional.sort_values(by=['sku','fecha'], ascending=True, inplace=True)
        
        # Agrupacion además de sku
        else:
            transaccional = df.groupby([pd.Grouper(key='fecha', freq='W-MON'), agrupacion,'sku']).sum()
            transaccional.reset_index(drop=False, inplace=True)
            transaccional.sort_values(by=[agrupacion,'sku','fecha'], ascending=True, inplace=True)
            transaccional["item"] = transaccional[agrupacion].astype(str) + '|' + transaccional["sku"].astype(str)
            del transaccional['sku']
            del transaccional[agrupacion]
        
    # Cambio de nombre de columnas y reordenamiento para coincidir con dataframe de inferencias   
    transaccional.rename(columns={'sku' : 'item', 'fecha': 'date'}, inplace= True)
    transaccional = transaccional[['date', 'item', 'cantidad']]
    
    # unión transaccional con inferencias
    merged=pd.merge(transaccional,inferencia, how='outer')
    merged.sort_values(by=['item','date'], ascending=True, inplace=True)
    del merged['model']
    merged = merged.rename(columns={'cantidad' : 'hist', 'units': 'pred'}).fillna(0)
    merged = merged[(merged.date <= max(inferencia['date'])) & (merged.date >= min(inferencia['date']))]
    
    # construcción de métricas
    smape = []
    accuracy = []
    mape = []
    bias = []
    hist_pred = []
    hist_pred2 = []
    
    for i in range(0,len(merged)):
        hist = merged.iloc[i,2].astype(int)
    #    hist = hist + 0.01
        pred = merged.iloc[i,3]

        hist_pred_i = ((hist) - pred)
        hist_pred_i2 = ((hist) - pred)**2
        hist_pred.append(hist_pred_i)
        hist_pred2.append(hist_pred_i2)

        if pred != 0:
            bias_i = round((((hist)/pred)-1),5)*100
        else:
            bias_i = 0
        
        bias.append(bias_i)

        if abs(hist_pred_i) > 0.05:
            smape_i = 100*((abs(hist_pred_i)/((abs(hist) + abs(pred))/2)))
        else:
            smape_i = 0
            
        smape.append(smape_i)
            
        if hist != 0:
            #mape_i = abs(round((((hist_pred_i)/(hist)),2)))
            accuracy_i = (1-(abs(hist_pred_i)/(hist)))*100
        else:
            #mape_i = 0
            accuracy_i = 0
        
       # mape.append(mape_i)
        accuracy.append(accuracy_i)
           
            
    # smape y mape
    merged['smape1'] = smape
    #merged['mape1'] = mape
    #merged['mape1'] = merged['mape1']
    
    # forecast accuracy
    merged['accuracy1'] = accuracy

    # bias
    merged['bias1'] = bias
    
    # diferencia absoluta entre historico e inferencia
    merged['hist_pred'] = hist_pred
    merged['rmse1'] = abs(merged['hist_pred'])
    merged['hist_pred2'] = hist_pred2
    del merged['hist_pred']
    
    merged = merged[['date', 'item', 'hist','pred','rmse','rmse1','smape','smape1','accuracy','accuracy1','bias','bias1','hist_pred2']]

    return merged

### Usar función para unir transaccional con inferencias + cálculo métricas, asignar a dataframe "inf"

In [ ]:
inf = metricas_error(trs,
                     'ruta de acceso archivo inferencia',
                     semanal=False,
                     ag2='tienda');inf

### Divide el dataframe "inf" y asigna el periodo de validación al dataframe "val"

In [ ]:
val = inf[inf['date'] <= 'presente-virtual-prediccion']

### Divide el dataframe "inf" y asigna el periodo de predicción al dataframe "pron"

In [ ]:
pron = inf[inf['date'] > 'presente-virtual-prediccion']

## Función: Sobrepronósticos y Subpronósticos en dataframe

In [ ]:
# se asigna datafame e índice de la columna con "bias1" para revisar cuántos items fueron sobre o sub pronosticados

quiebre = []
exceso = []

for i in range(0,len(pron)):
    bias_i = pron.iloc[i,11]
    
    if bias_i > 0:
        quiebre.append(bias_i)
    else:
        exceso.append(bias_i)

nq = len(quiebre)
ne = len(exceso)

print(f'Numero de items sobrepronosticados: {ne}\n'
     f'Numero de items subpronosticados: {nq}')

### Usar función para mostrar total de predicciones por arriba y debajo, junto con %

In [ ]:
nivel_stock(inf,10)

## Función: Resumen Inferencias

In [ ]:
# Para calcular a nivel de sku o ventana de tiempo smape promedio, bias y forecast accuracy

def resumen_inferencias(df=df2, grupo='item'):
    
    #resultados = df.where(df['pred_hist'] != 0).groupby([grupo]).agg({'hist':'sum','pred':'sum','pred_hist':'sum','pred_hist2':np.mean}).reset_index()
    resultados = df.groupby([grupo]).agg({'hist':'sum','pred':'sum','rmse':np.mean,'rmse1':'sum','accuracy':np.mean,'hist_pred2':np.mean}).reset_index()
    #resultados['smape'] = df.groupby([grupo]).agg({'smape':np.mean}).reset_index().drop([grupo], axis=1).fillna(0)
    resultados['smape'] = round(df.groupby([grupo]).agg({'smape':np.mean}),2).reset_index().drop([grupo], axis=1)
    resultados['smape1'] = round(df.groupby([grupo]).agg({'smape1':np.mean}),2).reset_index().drop([grupo], axis=1)
    #resultados['mape1'] = round(df.groupby([grupo]).agg({'mape1':np.mean}).reset_index().drop([grupo], axis=1),3).fillna(0)
   # resultados['mape_0'] = round(df.where(df['mape1'] != 0).groupby([grupo]).agg({'mape1':np.mean}).reset_index().drop([grupo], axis=1),3).fillna(0)
   # resultados['weighted_mape'] = round(((resultados['pred_hist'] / resultados['hist']) - 1),2).replace([np.inf, -np.inf], 0).fillna(0)
    #resultados['rmse'] = df.groupby([grupo]).agg({'rmse':np.mean}).reset_index().drop([grupo], axis=1)
    resultados['hist_pred_abs'] = resultados['rmse1']
    resultados['rmse1'] = round((resultados['hist_pred2']**0.5),2)
    #resultados['accuracy'] = df.groupby([grupo]).agg({'accuracy':np.mean}).reset_index().drop([grupo], axis=1)
    #resultados['accuracy'] = round((1 - (resultados['rmse1'] / resultados['hist'])),2).replace([np.inf, -np.inf], 4)
    resultados['accuracy1'] = round((1 - (resultados['hist_pred_abs'] / resultados['hist'])),2).replace([np.inf, -np.inf], 4)
    #resultados['bias'] = df.groupby([grupo]).agg({'bias':np.mean}).reset_index().drop([grupo], axis=1)
    resultados['bias'] = round(((resultados['hist'] / resultados['pred']) - 1),2)
    resultados['hist_pred2']
    
    resultados = resultados[[grupo, 'hist','pred','hist_pred_abs','rmse','rmse1','smape','smape1','accuracy','accuracy1','bias','hist_pred2']]

    return resultados

### Usar función para calcular métricas a nivel de agrupación

In [ ]:
item = resumen_inferencias(val);item

In [6]:
mes_o_semana = resumen_inferencias(val,grupo='date');mes_o_semana

NameError: name 'resumen_inferencias' is not defined

## Función: Métricas Globales

In [ ]:
def metricas_globales(df):
    sum_hist = df['hist'].sum()
    sum_pred = df['pred'].sum()
    sum_hist_pred = df['hist_pred_abs'].sum()
    mean_hist_pred2 = df['hist_pred2'].mean()
    hist_pred2 = []
    
    for i in range(0,len(df)):
        hist = df.iloc[i,1]
        pred = df.iloc[i,2]
        
        dif = (hist - pred)**2
        hist_pred2.append(dif)
    
    rmse = round((sum(hist_pred2) / len(hist_pred2))**0.5,2)
    rmse1 = round(df['rmse'].mean(),2)
    smape = round(df['smape'].mean(),3)
    accuracy = round(1 - (sum_hist_pred/sum_hist),3)*100
    bias = round((sum_hist/sum_pred) - 1,3)
    
    print(f'RMSE = {rmse} ; {rmse1}\n'
         f'SMAPE = {smape}\n'
         f'FA = {accuracy}%\n'
         f'Bias = {bias}')

### Usar función para calcular métricas a nivel de modelo (globales)

In [ ]:
metricas_globales(item)

### Graficar SMAPE para periodo validación

In [ ]:
# Grafico smape
sns.set(rc={'figure.figsize':(15,10)})
sns.histplot(data=val['smape1'], kde=True,binwidth=20)

### Graficar SMAPE para periodo pronósticos

In [ ]:
# Grafico smape 2
sns.histplot(data=pron['smape1'], kde=True,binwidth=20)

### Graficar Forecast Accuracy por ventana de tiempo

In [ ]:
# Grafico forecast accuracy
sns.set(rc={'figure.figsize':(15,10)})
sns.lineplot(data=mes.drop(columns=['rmse','smape','bias','hist_pred2','hist_pred_abs','hist','pred'], axis=1).set_index('date'))

### Graficar ventas vs predicciones por ventana de tiempo

In [ ]:
# Grafico venta vs prediccion
sns.set(rc={'figure.figsize':(15,10)})
sns.lineplot(data=mes.drop(columns=['rmse','smape','accuracy','bias','hist_pred2','hist_pred_abs'], axis=1).set_index('date'))

### Graficar venta promedio vs predicción promedio por ventana de tiempo + intervalos 95%

In [ ]:
# Grafico venta promedio vs inferencia promedio + intervalos 95%
sns.lineplot(data=inf.drop(columns=['rmse','smape','accuracy','bias','hist_pred2','rmse1','smape1','accuracy1','item','bias','bias1'], axis=1).set_index('date'))

### Graficar RMSE vs SMAPE vs Volumen ventas

In [ ]:
# Grafico rmse vs smape vs volumen
fig, ax1 = plt.subplots()
ax1 = item[item['rmse1'] <=5000].plot.scatter(x='smape1', y='rmse', c='hist', colormap='plasma',figsize=(20,15), legend = True,logy=True, ax=ax1)
ax1.set_xlabel("smape")
plt.show()